<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>1D ConvNet on Sentiment Analysis</H1></u></center>

# First run this from terminal

In [2525]:
# import nltk
# nltk.download('stopwords')

In [2526]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [2527]:
#import time
#start_time = time.time()
import pandas as pd
import numpy as np

In [2528]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

In [2529]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import Dense, Embedding, LSTM, CuDNNLSTM, Dropout, Flatten, Input
from keras.datasets import imdb

In [2530]:
num_words = 50000

In [2531]:
# def load_data(path='imdb.npz', num_words=None, skip_top=0,
#               maxlen=None, seed=113,
#               start_char=1, oov_char=2, index_from=3, **kwargs):

#     with np.load(path) as f:
#         x_train, labels_train = f['x_train'], f['y_train']
#         x_test, labels_test = f['x_test'], f['y_test']

#     np.random.seed(seed)
#     indices = np.arange(len(x_train))
#     np.random.shuffle(indices)
#     x_train = x_train[indices]
#     labels_train = labels_train[indices]

#     indices = np.arange(len(x_test))
#     np.random.shuffle(indices)
#     x_test = x_test[indices]
#     labels_test = labels_test[indices]

#     xs = np.concatenate([x_train, x_test])
#     labels = np.concatenate([labels_train, labels_test])

#     if start_char is not None:
#         xs = [[start_char] + [w + index_from for w in x] for x in xs]
#     elif index_from:
#         xs = [[w + index_from for w in x] for x in xs]

#     if maxlen:
#         xs, labels = _remove_long_seq(maxlen, xs, labels)
#         if not xs:
#             raise ValueError('After filtering for sequences shorter than maxlen=' +
#                              str(maxlen) + ', no sequence was kept. '
#                              'Increase maxlen.')
#     if not num_words:
#         num_words = max([max(x) for x in xs])

#     if oov_char is not None:
#         xs = [[w if (skip_top <= w < num_words) else oov_char for w in x]
#               for x in xs]
#     else:
#         xs = [[w for w in x if skip_top <= w < num_words]
#               for x in xs]

#     idx = len(x_train)
#     x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
#     x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])

#     return (x_train, y_train), (x_test, y_test)

In [2532]:
#(train_data, train_labels), (test_data, test_labels) = load_data(path='imdb.npz',num_words=10000)

In [2533]:
# (X_train, y_train), (X_test, y_test) = load_data(path='imdb.npz',num_words=10000)

In [2534]:
df = pd.read_csv('consumer_complaints_small.csv')

In [2535]:
df.columns

Index(['product', 'consumer_complaint_narrative'], dtype='object')

In [2536]:
# X = 'consumer_complaint_narrative', Y = 'product'

df = df[['consumer_complaint_narrative','product']]

In [2537]:
df.head()

,consumer_complaint_narrative,product
0,XXXX has claimed I owe them {$27.00} for XXXX ...,Debt collection
1,Due to inconsistencies in the amount owed that...,Consumer Loan
2,In XX/XX/XXXX my wages that I earned at my job...,Mortgage
3,I have an open and current mortgage with Chase...,Mortgage
4,XXXX was submitted XX/XX/XXXX. At the time I s...,Mortgage


In [2538]:
len(df)

66806

In [2539]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66806 entries, 0 to 66805
Data columns (total 2 columns):
consumer_complaint_narrative    66806 non-null object
product                         66806 non-null object
dtypes: object(2)
memory usage: 1.0+ MB


In [2540]:
df['product'].value_counts()

Debt collection            17552
Mortgage                   14919
Credit reporting           12526
Credit card                 7929
Bank account or service     5711
Consumer Loan               3678
Student loan                2128
Prepaid card                 861
Payday loan                  726
Money transfers              666
Other financial service      110
Name: product, dtype: int64

In [2541]:
df['product'].value_counts().sort_values(ascending=False).iplot(kind='bar', yTitle='Number of Reviews', 
                                                                title='Number reviews in each rating')

## Down-sampling the data

In [2542]:
# # Using Ramdom Under Sampling (RUS) technique
# X = df.iloc[:, :1].values
# y = df.loc[:, 'Score'].values

# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(return_indices=True)
# X_rus, y_rus, id_rus = rus.fit_sample(X, y)

In [2543]:
# X_rus[:,0]

In [2544]:
#y_rus[:,1]

In [2545]:
# X_rus.shape

In [2546]:
# X_rus[10]

In [2547]:
# len(X)

In [2548]:
# len(X_rus)

In [2549]:
# id_rus

In [2550]:
# X_rus = X_rus[:,0]

In [2551]:
# X_rus.shape

In [2552]:
# y_rus

In [2553]:
# y_rus.shape

In [2554]:
##y_rus = y_rus[:,0]

In [2555]:
# df = pd.DataFrame({'Text':X_rus,'Score':y_rus})

## df = pd.DataFrame({'x':x, 'y':y})

In [2556]:
# df['Text'][1]

In [2557]:
def print_plot(index):
    example = df[df.index == index][['consumer_complaint_narrative', 'product']].values[0]
    if len(example) > 0:
        print(example[0])
        print('product:', example[1])

In [2558]:
print_plot(10)

I am very disappointed that the CFPB did not help to resolve this fraudulant loan. # XXXX between XXXX XXXX and XXXX, FF ( AKA ) One West. Case # XXXX with CFPB. 

product: Mortgage


In [2559]:
print_plot(100)

I contacted Drivetime on several different occasions regarding a car loan I had with them account # XXXX i do not have the number for the other loan that was removed also from the credit bureau with a positive credit rating that was closed and paid in full. The company insisted on avoiding an agreement entered into by myself and XXXX of their company representative in a corporate position they have the agreed upon resolution and can provide your agency with a copy of the same because i do not have the capability to do so with my computer. Drivetime during the month of XXXX XXXX removed my credit reporting information from XXXX credit bureaus reporting agencies I believe XXXX and XXXX XXXX I discovered in or about XXXX XXXX that my positive credit ratings were no longer in the XXXX and XXXX credit bureaus and that another positive account for another car loan was no longer reporting in my credit file. these removals adversely affecxted my credit ratings and as such my ability to purchas

In [2560]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].apply(clean_text)

In [2561]:
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].str.replace('\d+', '')

In [2562]:
print_plot(10)

disappointed cfpb help resolve fraudulant loan # ff aka one west case # cfpb
product: Mortgage


In [2563]:
print_plot(100)

contacted drivetime several different occasions regarding car loan account # number loan removed also credit bureau positive credit rating closed paid full company insisted avoiding agreement entered company representative corporate position agreed upon resolution provide agency copy capability computer drivetime month removed credit reporting information credit bureaus reporting agencies believe discovered positive credit ratings longer credit bureaus another positive account another car loan longer reporting credit file removals adversely affected credit ratings ability purchase something lower interest rate never missed payment drivetime company car loans inclusive different auto loans destroy credit ratings despicable spoke today regarding issues response drivetime process updating consumers credit files consumers credit files removed credit bureaus updated however know process performed records updated corrected process timeframes process take furthermore process involves consumer

# ==================================

## Test- only first 5000 observations

In [2564]:
len(df)

66806

In [2565]:
df.head()

,consumer_complaint_narrative,product
0,claimed owe years despite proof payment sent ...,Debt collection
1,due inconsistencies amount owed told bank amou...,Consumer Loan
2,wages earned job decreased almost half knew tr...,Mortgage
3,open current mortgage chase bank # chase repor...,Mortgage
4,submitted time submitted complaint dealt rushm...,Mortgage


# Shuffle the dataframe index, inplace

In [2566]:
# df = df.sample(frac=1).reset_index(drop=True)

In [2567]:
#df = df[:5000]

In [2568]:
df.head()

,consumer_complaint_narrative,product
0,claimed owe years despite proof payment sent ...,Debt collection
1,due inconsistencies amount owed told bank amou...,Consumer Loan
2,wages earned job decreased almost half knew tr...,Mortgage
3,open current mortgage chase bank # chase repor...,Mortgage
4,submitted time submitted complaint dealt rushm...,Mortgage


In [2569]:
df['product'].value_counts()

Debt collection            17552
Mortgage                   14919
Credit reporting           12526
Credit card                 7929
Bank account or service     5711
Consumer Loan               3678
Student loan                2128
Prepaid card                 861
Payday loan                  726
Money transfers              666
Other financial service      110
Name: product, dtype: int64

# ====================================

In [2570]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['consumer_complaint_narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 56802 unique tokens.


In [2571]:
X = tokenizer.texts_to_sequences(df['consumer_complaint_narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (66806, 250)


In [2572]:
Y = pd.get_dummies(df['product']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (66806, 11)


In [2573]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(53444, 250) (53444, 11)
(13362, 250) (13362, 11)


# finished data preprocessing

# ====================================================================

# start building CNN 

In [2574]:
print(len(X_train), 'train_data')
print(len(X_test), 'test_data')

53444 train_data
13362 test_data


In [2575]:
#(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [2576]:
max_len = 256
embedding_size = 100
batch_size = 128

## Sequences per time

In [2577]:
print(X_train[2])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     1    14     2   313    47    20    35    48  3114   313
    47    10     2    13     1   221    45     5    35    48   868    48
   649     2   601  1012    20    33    43  1428    33    65     5   574
   313    47   590     5   320     5   110   100     6     5   274  1301
  2612   282   624   561     1    10   153   313    47   601    10  1012
    43   274     5    44   216   195   601    52     2   759    48  1012
    20    16   820   227  3225   590   530  1013   

In [2578]:
len(X_train[1])

250

In [2579]:
pad  =  'post' #'pre'
#Convert our lists to equal length sequences
X_train_pad = pad_sequences(X_train, maxlen=max_len, padding=pad, truncating=pad)
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding=pad, truncating=pad)

In [2580]:
X_train_pad.shape

(53444, 256)

In [2581]:
X_train_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [2582]:
X_train_pad[4499]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [2583]:
#print("--- %s seconds ---" % (time.time() - start_time))

## Creating the model (Model 0):

In [2584]:
model0 = Sequential()

In [2585]:
model0.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [2586]:
model0.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))

In [2587]:
model0.add(MaxPooling1D(pool_size=5))

In [2588]:
model0.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))

In [2589]:
model0.add(MaxPooling1D(pool_size=5))

In [2590]:
model0.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))

In [2591]:
# l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
# l_pool1 = MaxPooling1D(5)(l_cov1)
# l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
# l_pool2 = MaxPooling1D(5)(l_cov2)
# l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
# l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
# l_flat = Flatten()(l_pool3)
# l_dense = Dense(128, activation='relu')(l_flat)
# preds = Dense(2, activation='softmax')


In [2592]:
# model0.add(MaxPooling1D(pool_size=5))
model0.add(MaxPooling1D(2))
# model0.add(MaxPooling1D(pool_size=2))

In [2593]:
model0.add(Dropout(0.2))

In [2594]:
model0.add(Flatten())

In [2595]:
model0.add(Dense(250, activation='relu'))

In [2596]:
#model0.add(Dense(5, activation='softmax', name='classification'))

model0.add(Dense(11, activation='softmax'))

## Compiling the model:

In [2597]:
model0.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [2598]:
model0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 100)          5000000   
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 256, 128)          64128     
_________________________________________________________________
max_pooling1d_83 (MaxPooling (None, 51, 128)           0         
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 51, 128)           82048     
_________________________________________________________________
max_pooling1d_84 (MaxPooling (None, 10, 128)           0         
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 10, 128)           82048     
_________________________________________________________________
max_pooling1d_85 (MaxPooling (None, 5, 128)            0         
__________

## Training

In [2599]:
#%%time
model0.fit(X_train_pad, Y_train, validation_split=0.15, epochs=4, batch_size=batch_size)

Train on 45427 samples, validate on 8017 samples
Epoch 1/4
45427/45427 [==============================] - 55s 1ms/step - loss: 1.0374 - acc: 0.6513 - val_loss: 0.6758 - val_acc: 0.7886
Epoch 2/4
45427/45427 [==============================] - 42s 930us/step - loss: 0.5611 - acc: 0.8308 - val_loss: 0.5132 - val_acc: 0.8468
Epoch 3/4
45427/45427 [==============================] - 42s 933us/step - loss: 0.4334 - acc: 0.8714 - val_loss: 0.5046 - val_acc: 0.8458
Epoch 4/4
45427/45427 [==============================] - 42s 934us/step - loss: 0.3413 - acc: 0.8972 - val_loss: 0.4962 - val_acc: 0.8533


## Testing

In [2600]:
#%%time
eval_ = model0.evaluate(X_test_pad, Y_test)

13362/13362 [==============================] - 4s 313us/step


In [2601]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

Loss: 0.52556
Accuracy: 84.77%


# ===============================================

# Now show the F-1 score and Accuracy

In [2602]:
accr = model0.evaluate(X_test_pad,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))



13362/13362 [==============================] - 4s 316us/step
Test set
  Loss: 0.526
  Accuracy: 0.848


In [2603]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2604]:
def decode(datum):
    return np.argmax(datum)

In [2605]:
# for i in range(Y_test.shape[0]):
#     datum = Y_test[i]
#     print('index: %d' % i)
#     print('encoded datum: %s' % datum)
#     decoded_datum = decode(Y_test[i])
#     print('decoded datum: %s' % decoded_datum)
#     print()

In [2606]:
Y_test = np.array([decode(i) for i in Y_test])

In [2607]:
yhat_probs = model0.predict(X_test_pad, verbose=0)
# predict crisp classes for test set
yhat_classes = model0.predict_classes(X_test_pad, verbose=0)

In [2608]:
yhat_classes

array([4, 6, 2, ..., 4, 3, 3])

In [2609]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(Y_test, yhat_classes)
print('Accuracy: %f' % accuracy)

Accuracy: 0.847702


In [2610]:
f1_score(Y_test, yhat_classes, average='macro') 

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



0.6716274344081145

In [2611]:
f1_score(Y_test, yhat_classes, average='micro') 

0.8477024397545277

In [2612]:
f1_score(Y_test, yhat_classes, average=None) 

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



array([0.80332168, 0.70596299, 0.80230978, 0.86055154, 0.86019444,
       0.5751073 , 0.93855121, 0.        , 0.23931624, 0.76404494,
       0.83854167])

In [2613]:
f1_score(Y_test, yhat_classes, average='weighted') 

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



0.8434418391335528

In [2614]:
unique_elements, counts_elements = np.unique(Y_test, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1    2    3    4    5    6    7    8    9   10]
 [1155  746 1615 2502 3496  150 2941   21  153  168  415]]


In [2615]:
unique_elements, counts_elements = np.unique(yhat_classes, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1    2    3    4    5    6    8    9   10]
 [1133  713 1329 2611 3807   83 3064   81  188  353]]


In [2616]:
# confusion matrix
matrix = confusion_matrix(Y_test, yhat_classes)
print(matrix)

[[ 919   13   53   27   46    7   61    0    2   25    2]
 [   6  515   19   60   91    0   45    0    9    0    1]
 [ 100   30 1181  101  156    1   25    0    0   21    0]
 [   9   27   23 2200  214    0   23    0    1    0    5]
 [  18   44   40  148 3141    2   60    0   21    2   20]
 [  43    3    3    1   12   67   10    0    6    4    1]
 [   7   22    3   56   35    0 2818    0    0    0    0]
 [   2    1    2    1    7    3    2    0    1    0    2]
 [   7   48    0    1   63    0    6    0   28    0    0]
 [  20    0    5    1    5    1    0    0    0  136    0]
 [   2   10    0   15   37    2   14    0   13    0  322]]


# ==================================

## Model 1:

In [600]:
model1 = Sequential()

In [601]:
model1.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [602]:
model1.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))

In [603]:
model1.add(MaxPooling1D(pool_size=3))

In [604]:
model1.add(CuDNNLSTM(128))

KeyboardInterrupt: 

In [ ]:
model1.add(Dropout(0.5))

In [ ]:
model1.add(Dense(1, activation='sigmoid', name='classification'))

## Compiling the model:

In [ ]:
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()

## Training the model:

In [ ]:
%%time
model1.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

## Testing the model:

In [ ]:
%%time
eval_ = model1.evaluate(X_test_pad, y_test)

In [ ]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

## Conv1D model with multiple kernel sizes:

In [ ]:
from keras.layers import concatenate

In [ ]:
conv = []
filter_sizes = [3,4,5]

In [ ]:
embedding_layer = Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding')

In [ ]:
seq_input = Input(shape=(max_len,), dtype='int32')
embed_seq = embedding_layer(seq_input)

for f in filter_sizes:
    _conv = Conv1D(filters=128,kernel_size=f,activation='relu')(embed_seq)
    _pool = MaxPooling1D(5)(_conv)
    conv.append(_pool)

In [ ]:
_concat = concatenate(conv, axis=1)
_conv1= Conv1D(128, 5, activation='relu')(_concat)
_pool1 = MaxPooling1D(5)(_conv1)
_pool1 = Dropout(0.5)(_pool1)
_conv2 = Conv1D(128, 5, activation='relu')(_pool1)
_pool2 = MaxPooling1D(5)(_conv2)
_flat = Flatten()(_pool2)
_flat = Dropout(0.5)(_flat)
_dense = Dense(128, activation='relu')(_flat)
preds = Dense(1, activation='sigmoid')(_dense)

In [ ]:
model = Model(seq_input, preds)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [ ]:
%%time
model.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

In [ ]:
%%time
eval_ = model.evaluate(X_test_pad, y_test)

In [ ]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

In [ ]:
#print("Total time used....")
#print("--- %s seconds ---" % (time.time() - start_time))

## Reference:

https://keras.io/layers/convolutional/